# Phi-4 Mini ONNX समानान्तर फङ्सन कलिङ ट्युटोरियल

यो नोटबुकले Phi-4 Mini लाई ONNX Runtime GenAI सँग समानान्तर फङ्सन कलिङको लागि कसरी प्रयोग गर्ने भनेर देखाउँछ। फङ्सन कलिङले मोडेललाई प्रयोगकर्ताको अनुरोधको आधारमा बाह्य उपकरणहरू र API हरूलाई बुद्धिमानी रूपमा बोलाउन अनुमति दिन्छ।

## अवलोकन

यस ट्युटोरियलमा, तपाईंले सिक्नुहुनेछ:
- Phi-4 Mini लाई ONNX Runtime GenAI सँग सेट अप गर्ने
- उडान र होटल बुकिङको लागि फङ्सन स्किमाहरू परिभाषित गर्ने
- संरचित आउटपुटको लागि Lark व्याकरणको साथ निर्देशित जेनेरेसन प्रयोग गर्ने
- जटिल यात्रा बुकिङ परिदृश्यहरूको लागि समानान्तर फङ्सन कलहरू कार्यान्वयन गर्ने

## पूर्वआवश्यकताहरू

यो नोटबुक चलाउनु अघि, सुनिश्चित गर्नुहोस् कि तपाईंले:
- Phi-4 Mini ONNX मोडेल डाउनलोड गर्नुभएको छ
- `onnxruntime-genai` प्याकेज स्थापना गर्नुभएको छ
- फङ्सन कलिङ अवधारणाहरूको आधारभूत समझ


## चरण १: आवश्यक लाइब्रेरीहरू आयात गर्नुहोस्

पहिले, हामी हाम्रो फङ्सन कलिङ कार्यान्वयनको लागि आवश्यक लाइब्रेरीहरू आयात गर्नेछौं।


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## चरण २: मोडेल सेटअप र कन्फिगरेसन

अब हामी Phi-4 Mini ONNX मोडेललाई कन्फिगर गर्नेछौं। आफ्नो वास्तविक मोडेल निर्देशिका प्रयोग गर्न पथलाई परिवर्तन गर्न निश्चित गर्नुहोस्।


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## चरण ३: उत्पादन प्यारामिटरहरू कन्फिगर गर्नुहोस्

मोडेलको आउटपुट व्यवहार नियन्त्रण गर्न उत्पादन प्यारामिटरहरू सेट गर्नुहोस्। यी सेटिङहरूले फङ्सन कलिङका लागि निश्चित र केन्द्रित प्रतिक्रिया सुनिश्चित गर्छ।


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## चरण ४: उपलब्ध कार्यहरू परिभाषित गर्नुहोस्

यहाँ हामी हाम्रो AI सहायकले कल गर्न सक्ने कार्यहरू परिभाषित गर्छौं। यस उदाहरणमा, हामीसँग दुई यात्रा सम्बन्धित कार्यहरू छन्:
1. **booking_flight_tickets**: विमानस्थलहरू बीच उडान टिकट बुक गर्नका लागि
2. **booking_hotels**: होटल आवास बुक गर्नका लागि

कार्य परिभाषाहरू OpenAI को कार्य कल गर्ने स्कीमा ढाँचा अनुसरण गर्छ।


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## चरण ५: व्याकरण निर्माण सहायक कार्यहरू

यी सहायक कार्यहरूले हाम्रो कार्य परिभाषालाई Lark व्याकरण ढाँचामा रूपान्तरण गर्छन्, जुन ONNX Runtime GenAI द्वारा निर्देशित निर्माणको लागि प्रयोग गरिन्छ। यसले मोडेलले सही JSON ढाँचामा मान्य कार्य कलहरू उत्पादन गर्न सुनिश्चित गर्दछ।


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## चरण ६: व्याकरण निर्माण परीक्षण गर्नुहोस्

आउनुहोस्, हाम्रो व्याकरण निर्माण कार्यहरू परीक्षण गरौं ताकि तिनीहरूले हाम्रो उपकरण परिभाषाहरूलाई सही ढाँचामा कसरी रूपान्तरण गर्छन् भन्ने हेर्न सकियोस्।


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## चरण ७: प्रणाली प्रम्प्ट तयार गर्नुहोस् र जेनेरेटर सेटअप गर्नुहोस्

अब हामी प्रणाली प्रम्प्ट तयार गर्नेछौं जसले मोडेललाई उपलब्ध उपकरणहरूको बारेमा जानकारी दिन्छ र निर्देशित उत्पादन मापदण्डहरूसँग जेनेरेटर सेटअप गर्नेछौं।


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## चरण ८: निर्देशित उत्पादनको लागि जेनरेटर सुरु गर्नुहोस्

अब हामीले कन्फिगर गरेका प्यारामिटरहरू प्रयोग गरेर जेनरेटर बनाउनेछौं र निर्देशित उत्पादनको लागि Lark व्याकरण लागू गर्नेछौं।


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## चरण ९: समानान्तर फङ्सन कलिङ परीक्षण गर्नुहोस्

अब हामी हाम्रो सेटअपलाई जटिल यात्रा बुकिङ परिदृश्यसँग परीक्षण गरौं जसले धेरै फङ्सनहरू कल गर्न आवश्यक छ।


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### फंक्शन कलहरू उत्पन्न गर्नुहोस्

मोडेल अब प्रयोगकर्ताको अनुरोधको आधारमा संरचित फंक्शन कलहरू उत्पन्न गर्नेछ। निर्देशित उत्पादनको कारण, नतिजा मान्य JSON ढाँचामा हुनेछ जसलाई सिधै कार्यान्वयन गर्न सकिन्छ।

**अपेक्षित नतिजा**: मोडेलले निम्न फंक्शन कलहरू उत्पन्न गर्नुपर्छ:
1. `booking_flight_tickets` बीजिङ (PEK) देखि पेरिस (CDG) विवरणसहित
2. `booking_hotels` पेरिसको आवास विवरणसहित

तलको सेल चलाएर प्रत्यक्ष उत्पादन हेर्नुहोस्:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## निष्कर्ष

🎉 **बधाई छ!** तपाईंले ONNX Runtime GenAI प्रयोग गरेर Phi-4 Mini सँग समानान्तर फङ्सन कलिङ सफलतापूर्वक कार्यान्वयन गर्नुभएको छ।

### तपाईंले के सिक्नुभयो:

1. **मोडेल सेटअप**: ONNX Runtime GenAI सँग Phi-4 Mini कन्फिगर गर्ने तरिका
2. **फङ्सन परिभाषा**: AI फङ्सन कलिङका लागि फङ्सन स्किमाहरू परिभाषित गर्ने तरिका
3. **निर्देशित जेनेरेसन**: संरचित आउटपुट जेनेरेसनका लागि Lark व्याकरण प्रयोग गर्ने तरिका
4. **समानान्तर फङ्सन कलहरू**: धेरै फङ्सन कलहरू आवश्यक पर्ने जटिल अनुरोधहरू व्यवस्थापन गर्ने तरिका

### मुख्य फाइदाहरू:

- ✅ **संरचित आउटपुट**: निर्देशित जेनेरेसनले मान्य JSON फङ्सन कल सुनिश्चित गर्दछ
- ✅ **समानान्तर प्रशोधन**: एकल अनुरोधमा धेरै फङ्सन कलहरू व्यवस्थापन गर्नुहोस्
- ✅ **उच्च प्रदर्शन**: ONNX Runtime ले अनुकूलित इनफरेन्स प्रदान गर्दछ
- ✅ **लचिलो स्किमा**: फङ्सन परिभाषाहरू सजिलै थप्न वा परिमार्जन गर्न सकिन्छ

### स्रोतहरू:

- [Phi-4 Mini दस्तावेज](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI दस्तावेज](https://onnxruntime.ai/docs/genai/)
- [फङ्सन कलिङका उत्कृष्ट अभ्यासहरू](https://platform.openai.com/docs/guides/function-calling)



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको छ। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
